In [1]:
import platform
import numpy as np
import pandas as pd
import sklearn as sk
import os

In [17]:
df = pd.read_csv('C:\\Users\\DELL\\Desktop\\python Progs\\NBA_2013_2014_Games.csv')

In [18]:
from sklearn.metrics import f1_score, make_scorer, classification_report

scorer = make_scorer(f1_score, pos_label = None, average = 'weighted')

In [19]:
df.head()

,Date,Visitor,VisitorPts,Home,HomePts
0,10/30/2013,ORL,87,IND,97
1,10/30/2013,LAC,103,LAL,116
2,10/30/2013,CHI,95,MIA,107
3,10/30/2013,BRK,94,CLE,98
4,10/30/2013,ATL,109,DAL,118


In [20]:
df.dtypes

Date          object
Visitor       object
VisitorPts     int64
Home          object
HomePts        int64
dtype: object

In [22]:
df['Home win'] = df['VisitorPts'] < df['HomePts']
y_true = df['Home win'].values

In [23]:
from sklearn.metrics import f1_score

y_pred = [1] * len(y_true)
print('F1: {0:.4f}%'.format(f1_score(y_true, y_pred, pos_label = None, average = 'weighted') * 100))

F1: 42.6408%


In [14]:
df.shape

(1004, 12)

In [24]:
df.head()

,Date,Visitor,VisitorPts,Home,HomePts,Home win
0,10/30/2013,ORL,87,IND,97,True
1,10/30/2013,LAC,103,LAL,116,True
2,10/30/2013,CHI,95,MIA,107,True
3,10/30/2013,BRK,94,CLE,98,True
4,10/30/2013,ATL,109,DAL,118,True


In [25]:
n_games = df['Home win'].count()
n_homewins = df['Home win'].sum()
win_percentage = n_homewins / n_games

print('Home Win percentage: {0:.2f}%'.format(100 * win_percentage))

Home Win percentage: 58.05%


In [26]:
from sklearn.metrics import f1_score

y_pred = [1] * len(y_true)
print('F1: {0:.4f}%'.format(f1_score(y_true, y_pred, pos_label = None, average = 'weighted') * 100))

F1: 42.6408%


# More Features

In [27]:
df['Home Last Win'] = False
df['Visitor Last Win'] = False

In [29]:
from collections import defaultdict

won_last = defaultdict(int)
for index, row in df.iterrows():
    home_team = row['Home']
    visitor_team = row['Visitor']
    row['Home Last Win'] = won_last[home_team]
    row['Visitor Last Win'] = won_last[visitor_team]
    won_last[home_team] = row['Home win']
    won_last[visitor_team] = not row['Home win']
df.loc[0:4]

,Date,Visitor,VisitorPts,Home,HomePts,Home win,Home Last Win,Visitor Last Win
0,10/30/2013,ORL,87,IND,97,True,False,False
1,10/30/2013,LAC,103,LAL,116,True,False,False
2,10/30/2013,CHI,95,MIA,107,True,False,False
3,10/30/2013,BRK,94,CLE,98,True,False,False
4,10/30/2013,ATL,109,DAL,118,True,False,False


# Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 14)

In [32]:
from sklearn.model_selection import cross_val_score
X_previouswins = df[['Home Last Win', 'Visitor Last Win']].values
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)
print("Using just the last result from the home and visitor teams")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
F1: 42.6409%


In [41]:
df.loc[0:4]

,Date,Visitor,VisitorPts,Home,HomePts,Home win,Home Last Win,Visitor Last Win
0,10/30/2013,ORL,87,IND,97,True,False,False
1,10/30/2013,LAC,103,LAL,116,True,False,False
2,10/30/2013,CHI,95,MIA,107,True,False,False
3,10/30/2013,BRK,94,CLE,98,True,False,False
4,10/30/2013,ATL,109,DAL,118,True,False,False


In [42]:
X_winstreak = df[["Home Last Win", "Visitor Last Win", "Home Last Win", "Visitor Last Win"]].values
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_winstreak, y_true, scoring = scorer)
print("Using whether the home team is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
F1: 42.6409%


In [43]:
X_previouswins = df[['Home Last Win', 'Visitor Last Win']].values
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)
print("Using just the last result from the home and visitor teams")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
F1: 42.6409%


In [52]:
df["Home Last Win"] = 0
df["Visitor Last Win"] = 0
from collections import defaultdict
win = defaultdict(int)
for index, row in df.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    row["Home Last Win"] = win[home_team]
    row["Visitor Last Win"] = win[visitor_team]
    df.loc[index] = row
    if row["Home Last Win"]:
        win[home_team] += 1
        win[visitor_team] = 0
    else:
        win[home_team] = 0
        win[visitor_team] += 1

In [53]:
df.loc[0:4]

,Date,Visitor,VisitorPts,Home,HomePts,Home win,Home Last Win,Visitor Last Win
0,10/30/2013,ORL,87,IND,97,True,0,0
1,10/30/2013,LAC,103,LAL,116,True,0,0
2,10/30/2013,CHI,95,MIA,107,True,0,0
3,10/30/2013,BRK,94,CLE,98,True,0,0
4,10/30/2013,ATL,109,DAL,118,True,0,0


In [54]:
X_winstreak = df[["Home Last Win", "Visitor Last Win"]].values
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_winstreak, y_true, scoring = scorer)
print("Using whether the home team is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
F1: 51.6247%


In [55]:
df['Home'].values

array(['IND', 'LAL', 'MIA', ..., 'POR', 'SAC', 'SAS'], dtype=object)

In [56]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoding = LabelEncoder()
encoding.fit(df["Home"].values)

LabelEncoder()

In [57]:
home_teams = encoding.transform(df["Home"].values)
visitor_teams = encoding.transform(df["Visitor"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

In [58]:
X_teams.shape

(1230, 2)

In [59]:
print('Home:', X_teams[0:30])
print('Visitor:', X_teams[0,30:])

Home: [[11 21]
 [13 12]
 [15  4]
 [ 5  2]
 [ 6  0]
 [ 8 29]
 [ 9 13]
 [10  3]
 [17 21]
 [18 11]
 [19 16]
 [22 15]
 [23 24]
 [25  7]
 [26 14]
 [27  1]
 [28 20]
 [ 4 19]
 [12  9]
 [ 0 27]
 [ 1 16]
 [ 2 15]
 [ 3  5]
 [ 7 24]
 [10  6]
 [13 26]
 [14  8]
 [17 20]
 [21 18]
 [23 28]]
Visitor: []


In [60]:
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_teams, y_true, scoring = scorer)
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

F1: 57.7392%


# Random Forest Classifiers

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state = 14)
scores = cross_val_score(rfc, X_teams, y_true, scoring = scorer)
print("Using full team labels is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using full team labels is ranked higher
F1: 57.7966%


In [69]:
rfc = RandomForestClassifier(random_state = 14)
scores = cross_val_score(rfc, X_teams, y_true, scoring = scorer)
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

F1: 57.7966%


In [91]:
df_15 = pd.read_csv('C:\\Users\\DELL\\Desktop\\python Progs\\gamesWithInfo2016-17.csv')

In [92]:
df_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1006 non-null   int64  
 1   Home        1006 non-null   object 
 2   Away        1006 non-null   object 
 3   W_PCT       1006 non-null   float64
 4   REB         1006 non-null   float64
 5   TOV         1006 non-null   float64
 6   PLUS_MINUS  1006 non-null   float64
 7   OFF_RATING  1006 non-null   float64
 8   DEF_RATING  1006 non-null   float64
 9   TS_PCT      1006 non-null   float64
 10  Result      1006 non-null   int64  
 11  Date        1006 non-null   object 
dtypes: float64(7), int64(2), object(3)
memory usage: 94.4+ KB


In [93]:
df_15.head()

,Unnamed: 0,Home,Away,W_PCT,REB,TOV,PLUS_MINUS,OFF_RATING,DEF_RATING,TS_PCT,Result,Date
0,0,Philadelphia 76ers,Chicago Bulls,-1.764924,-3.739935,2.574073,-2.619013,-2.549887,0.993680,-0.201678,0,11/25/2016
1,1,Los Angeles Lakers,Golden State Warriors,-1.945127,-0.311661,0.980599,-2.601317,-1.762117,1.925254,-1.734430,0,11/25/2016
2,2,New York Knicks,Charlotte Hornets,-0.376305,0.000000,1.838624,-1.044066,-0.497539,1.179995,-0.322685,1,11/25/2016
3,3,Phoenix Suns,Minnesota Timberwolves,0.143102,-0.089046,-0.183862,-0.884802,-1.036539,0.186315,-0.726040,0,11/25/2016
4,4,Boston Celtics,San Antonio Spurs,-1.060015,0.000000,0.429012,-0.814018,-0.704847,0.372630,-0.645369,0,11/25/2016


In [89]:
df_15.head(7)

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
5,5,1950.0,Ed Bartels,F,24.0,NYK,2.0,NaN,NaN,NaN,...,0.667,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.0,4.0
6,6,1950.0,Ralph Beard,G,22.0,INO,60.0,NaN,NaN,NaN,...,0.762,NaN,NaN,NaN,233.0,NaN,NaN,NaN,132.0,895.0


In [77]:
rank_14 = pd.read_csv('C:\\Users\\DELL\\Desktop\\python Progs\\gamesWithInfo2017-18.csv')

In [79]:
rank_14.head()

,Unnamed: 0,Home,Away,W_PCT,REB,TOV,PLUS_MINUS,OFF_RATING,DEF_RATING,TS_PCT,Result,Date
0,0,San Antonio Spurs,Oklahoma City Thunder,0.558987,1.709397,-0.334956,-0.633105,-0.090581,0.870947,-0.305420,1,11/17/2017
1,1,Los Angeles Lakers,Phoenix Suns,0.486319,0.691899,1.563127,1.213451,-0.384970,-2.552777,-0.190887,0,11/17/2017
2,2,Chicago Bulls,Charlotte Hornets,-1.218591,-0.610499,2.902950,-1.547590,-2.196594,-0.450490,-1.450745,1,11/17/2017
3,3,Brooklyn Nets,Utah Jazz,-0.240364,0.610499,-2.121386,-0.281380,0.588778,1.261372,-0.152710,1,11/17/2017
4,4,Cleveland Cavaliers,LA Clippers,0.827301,-1.180298,1.004867,-0.562760,0.294389,1.501633,0.992615,1,11/17/2017


In [94]:
df_15.columns = ['id','Home','visitor','winning percent','Rebounds','TurnOver','Plus_Minus','Off_rating','Def_rating','TS_Percent','Result','Date']
df_15.head(7)

,id,Home,visitor,winning percent,Rebounds,TurnOver,Plus_Minus,Off_rating,Def_rating,TS_Percent,Result,Date
0,0,Philadelphia 76ers,Chicago Bulls,-1.764924,-3.739935,2.574073,-2.619013,-2.549887,0.993680,-0.201678,0,11/25/2016
1,1,Los Angeles Lakers,Golden State Warriors,-1.945127,-0.311661,0.980599,-2.601317,-1.762117,1.925254,-1.734430,0,11/25/2016
2,2,New York Knicks,Charlotte Hornets,-0.376305,0.000000,1.838624,-1.044066,-0.497539,1.179995,-0.322685,1,11/25/2016
3,3,Phoenix Suns,Minnesota Timberwolves,0.143102,-0.089046,-0.183862,-0.884802,-1.036539,0.186315,-0.726040,0,11/25/2016
4,4,Boston Celtics,San Antonio Spurs,-1.060015,0.000000,0.429012,-0.814018,-0.704847,0.372630,-0.645369,0,11/25/2016
5,5,Cleveland Cavaliers,Dallas Mavericks,3.725951,1.959013,-0.490300,2.866758,3.482772,0.217367,2.460470,1,11/25/2016
6,6,Orlando Magic,Washington Wizards,0.487607,1.246645,-0.429012,-0.796322,-1.699924,-1.024732,-1.613423,0,11/25/2016


In [95]:
rank_14.columns = ['id','Home','visitor','winning percent','Rebounds','TurnOver','Plus_Minus','Off_rating','Def_rating','TS_Percent','Result','Date']
rank_14.head(7)

,id,Home,visitor,winning percent,Rebounds,TurnOver,Plus_Minus,Off_rating,Def_rating,TS_Percent,Result,Date
0,0,San Antonio Spurs,Oklahoma City Thunder,0.558987,1.709397,-0.334956,-0.633105,-0.090581,0.870947,-0.305420,1,11/17/2017
1,1,Los Angeles Lakers,Phoenix Suns,0.486319,0.691899,1.563127,1.213451,-0.384970,-2.552777,-0.190887,0,11/17/2017
2,2,Chicago Bulls,Charlotte Hornets,-1.218591,-0.610499,2.902950,-1.547590,-2.196594,-0.450490,-1.450745,1,11/17/2017
3,3,Brooklyn Nets,Utah Jazz,-0.240364,0.610499,-2.121386,-0.281380,0.588778,1.261372,-0.152710,1,11/17/2017
4,4,Cleveland Cavaliers,LA Clippers,0.827301,-1.180298,1.004867,-0.562760,0.294389,1.501633,0.992615,1,11/17/2017
5,5,Indiana Pacers,Detroit Pistons,-1.380697,-1.139598,-0.669911,-0.984830,-0.090581,1.531666,0.572662,1,11/17/2017
6,6,Dallas Mavericks,Minnesota Timberwolves,-2.850833,-0.406999,-0.111652,-1.283796,-1.721043,-0.120131,-1.221680,0,11/17/2017
